## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key



# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cidreira,BR,-30.1811,-50.2056,48.76,63,6,18.16,clear sky
1,1,Hermanus,ZA,-34.4187,19.2345,61.81,46,66,7.29,broken clouds
2,2,Lasa,IT,46.6166,10.7002,65.28,79,0,4.99,clear sky
3,3,Puerto Ayora,EC,-0.7393,-90.3518,76.96,92,53,8.01,broken clouds
4,4,Hobart,AU,-42.8794,147.3294,40.86,78,40,17.27,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))



What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Puerto Ayora,EC,-0.7393,-90.3518,76.96,92,53,8.01,broken clouds
16,16,Avarua,CK,-21.2078,-159.7750,78.85,65,75,17.27,broken clouds
25,25,Kidal,ML,18.4411,1.4078,89.24,28,98,4.99,overcast clouds
31,31,Teknaf,BD,20.8624,92.3058,78.73,92,30,6.89,scattered clouds
35,35,Albany,US,42.6001,-73.9662,75.02,57,100,1.99,overcast clouds
36,36,Callaway,US,30.1530,-85.5699,84.97,75,0,10.36,clear sky
37,37,Vaini,TO,-21.2000,-175.2000,82.56,78,20,11.50,few clouds
38,38,Mahibadhoo,MV,3.7833,72.9667,82.99,75,100,11.54,overcast clouds
43,43,Tamiahua,MX,21.2667,-97.4500,86.61,64,12,14.61,few clouds
44,44,Labuhan,ID,-6.8844,112.2051,77.94,89,99,5.59,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()


City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
preferred_cities_df.count()

City_ID                214
City                   214
Country                214
Lat                    214
Lng                    214
Max Temp               214
Humidity               214
Cloudiness             214
Wind Speed             214
Current Description    214
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()



City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Puerto Ayora,EC,76.96,broken clouds,-0.7393,-90.3518,
16,Avarua,CK,78.85,broken clouds,-21.2078,-159.7750,
25,Kidal,ML,89.24,overcast clouds,18.4411,1.4078,
31,Teknaf,BD,78.73,scattered clouds,20.8624,92.3058,
35,Albany,US,75.02,overcast clouds,42.6001,-73.9662,
36,Callaway,US,84.97,clear sky,30.1530,-85.5699,
37,Vaini,TO,82.56,few clouds,-21.2000,-175.2000,
38,Mahibadhoo,MV,82.99,overcast clouds,3.7833,72.9667,
43,Tamiahua,MX,86.61,few clouds,21.2667,-97.4500,
44,Labuhan,ID,77.94,overcast clouds,-6.8844,112.2051,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")





Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df
clean_hotel_df.head()


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Puerto Ayora,EC,76.96,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
16,Avarua,CK,78.85,broken clouds,-21.2078,-159.7750,Paradise Inn
25,Kidal,ML,89.24,overcast clouds,18.4411,1.4078,NaN
31,Teknaf,BD,78.73,scattered clouds,20.8624,92.3058,Forestry Guesthouse
35,Albany,US,75.02,overcast clouds,42.6001,-73.9662,NaN


In [11]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and </dt><dd>{Max Temp} °F</dd>
"""


# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]


# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
 # 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 11b. Display the figure

fig


Figure(layout=FigureLayout(height='420px'))